# PDF파일 로드

In [1]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./PDFS/Concepts.pdf")
pages = loader.load_and_split()

In [20]:
pages[0:2]

[Document(page_content='The Concepts section helps you learn about the parts of the Kubernetes system and the\nabstractions Kubernetes uses to represent your cluster , and helps you obtain a deeper\nunderstanding of how Kubernetes works.\nOverview\nKubernetes is a portable, extensible, open source platform for managing containerized\nworkloads and services, that facilitates both declarative configuration and automation. It has a\nlarge, rapidly growing ecosystem. Kubernetes services, support, and tools are widely available.\nCluster Architecture\nThe architectural concepts behind Kubernetes.\nContainers\nTechnology for packaging an application along with its runtime dependencies.\nWorkloads\nUnderstand Pods, the smallest deployable compute object in Kubernetes, and the higher-level\nabstractions that help you to run them.\nServices, Load Balancing, and Networking\nConcepts and resources behind networking in Kubernetes.\nStorage\nWays to provide both long-term and temporary storage to P

# PDF내용 분리

In [3]:
from langchain.text_splitter import CharacterTextSplitter
text_spliter = CharacterTextSplitter(
  separator="\n",
  chunk_size=1000,
  chunk_overlap=100,
  length_function=len,
)

texts = text_spliter.split_documents(pages)

In [21]:
texts[0:2]

[Document(page_content='The Concepts section helps you learn about the parts of the Kubernetes system and the\nabstractions Kubernetes uses to represent your cluster , and helps you obtain a deeper\nunderstanding of how Kubernetes works.\nOverview\nKubernetes is a portable, extensible, open source platform for managing containerized\nworkloads and services, that facilitates both declarative configuration and automation. It has a\nlarge, rapidly growing ecosystem. Kubernetes services, support, and tools are widely available.\nCluster Architecture\nThe architectural concepts behind Kubernetes.\nContainers\nTechnology for packaging an application along with its runtime dependencies.\nWorkloads\nUnderstand Pods, the smallest deployable compute object in Kubernetes, and the higher-level\nabstractions that help you to run them.\nServices, Load Balancing, and Networking\nConcepts and resources behind networking in Kubernetes.\nStorage\nWays to provide both long-term and temporary storage to P

# Load Embedding

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

Embeddings loaded


# Save embeddings to vectorDB

In [6]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

In [18]:
query = "kubernetes annotations"
docs = db.similarity_search(query)
print(docs)

query = "Namespaces and DNS"
docs = db.similarity_search(query)
print(docs)

[Document(page_content='The kubernetes.io/  and k8s.io/  prefixes are reserved for Kubernetes core components.\nFor example, here\'s a manifest for a Pod that has the annotation imageregistry: https://\nhub.docker.com/  :\napiVersion : v1\nkind: Pod\nmetadata :\n  name : annotations-demo\n  annotations :\n    imageregistry : "https://hub.docker.com/"\nspec:\n  containers :\n  - name : nginx\n    image : nginx:1.14.2\n    ports :\n    - containerPort : 80• \n• \n• \n• \n• \n• \n•', metadata={'source': './PDFS/Concepts.pdf', 'page': 22}), Document(page_content='deployment, management, introspection, and the like.\nSyntax and character set\nAnnotations  are key/value pairs. Valid annotation keys have two segments: an optional prefix\nand name, separated by a slash ( /). The name segment is required and must be 63 characters or\nless, beginning and ending with an alphanumeric character ( [a-z0-9A-Z] ) with dashes ( -),\nunderscores ( _), dots ( .), and alphanumerics between. The prefix is 

# Use Chatbot from verctorDB

In [7]:
from langchain.llms import LlamaCpp
model = LlamaCpp(
  model_path="../llama-2-7b-chat.Q5_K_M.gguf",
  temperature=0.0,
  top_p=1,
  max_tokens=8192,
  verbose=True,
  n_ctx=4096,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32             

In [10]:
from langchain.prompts import PromptTemplate
system_prompt_template = """You're a Kubernetes expert. You understand the question and can generate a good kubernetes manifest.
Question: {question}"""

prompt = PromptTemplate(template=system_prompt_template, input_variables=["question"])

In [11]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [13]:
from langchain_core.runnables import RunnablePassthrough
chain = {"question": RunnablePassthrough()} | prompt | model | output_parser